### RAG Piplelines - Data INgestion to Vector DB Pipleline

In [5]:
import os
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pathlib import Path

In [ ]:
### Reaad all the pdf's insode the directory 

def process_all_pdfs(pdf_directory):
    """Process all PDF files in a directory"""
    all_documents = []
    pdf_dir = Path(pdf_directory)

    #Find all PDF files recursively
    pdf_files = list(pdf_dir.rglob("**/*.pdf"))

    print(f"Found {len(pdf_files)} PDF files to process.")

    for pdf_file in pdf_files:
        print(f"Processing file: {pdf_file}")
        try:
            loader = PyMuPDFLoader(str(pdf_file))
            documents = loader.load()
            
            # Add source information to metadata
            for doc in documents:
                doc.metadata["source_file"] = pdf_file.name
                doc.metadata["source_typr"] = 'pdf'

            all_documents.extend(documents)
            print(f" Loaded {len(documents)} pages")
        
        except Exception as e:
            print(f" Error loading: {e}")
    print(f"\nTotal documents loaded: {len(all_documents)}")
    return all_documents

# Process all PDFs in the data directory
all_pdf_documnets = process_all_pdfs("../data")

Found 2 PDF files to process.
Processing file: ..\data\pdf_files\Al Rouf.pdf
 Loaded 3 pages
Processing file: ..\data\pdf_files\Product & Price Guide.pdf
 Loaded 5 pages

Total documents loaded: 8


In [10]:
all_pdf_documnets

[Document(metadata={'producer': 'Microsoft® Word for Microsoft 365', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2025-10-31T16:05:18+05:30', 'source': '..\\data\\pdf_files\\Al Rouf.pdf', 'file_path': '..\\data\\pdf_files\\Al Rouf.pdf', 'total_pages': 3, 'format': 'PDF 1.7', 'title': '', 'author': 'Saad Fahim', 'subject': '', 'keywords': '', 'moddate': '2025-10-31T16:05:18+05:30', 'trapped': '', 'modDate': "D:20251031160518+05'30'", 'creationDate': "D:20251031160518+05'30'", 'page': 0, 'source_file': 'Al Rouf.pdf', 'source_typr': 'pdf'}, page_content="Al Rouf Lighting Technology Co. Ltd. \nOfficial Company Profile (Internal Document) \nPage 1: About Al Rouf & Our Identity \nOur Company \nEstablished in 2014, Al Rouf Lighting Technology Co. Ltd. is a leading, ISO 9001-2015 QMS \nand ISO 45001:2018 HSE certified company based in the Kingdom of Saudi Arabia. We are a \npremier provider of comprehensive, turnkey lighting solutions. \nOur commitment to quality and innova

In [15]:
### Text splitting get into chunks

def split_documents(documents, chunk_size=1000, chunk_overlap=200):
    """Split documnets into smaller chunks for better RAG performance"""

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )
    split_docs = text_splitter.split_documents(documents)
    print(f"Split  {len(documents)}  documents into {len(split_docs)} chunks")

    # show example of a chunk
    if split_docs:
        print("\nExample chunk:")
        print(f"Content: {split_docs[0].page_content[:200]}...")
        print(f"Metadata: {split_docs[0].metadata}")
        
    return split_docs

In [16]:
chunks = split_documents(all_pdf_documnets)
chunks

Split  8  documents into 13 chunks

Example chunk:
Content: Al Rouf Lighting Technology Co. Ltd. 
Official Company Profile (Internal Document) 
Page 1: About Al Rouf & Our Identity 
Our Company 
Established in 2014, Al Rouf Lighting Technology Co. Ltd. is a le...
Metadata: {'producer': 'Microsoft® Word for Microsoft 365', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2025-10-31T16:05:18+05:30', 'source': '..\\data\\pdf_files\\Al Rouf.pdf', 'file_path': '..\\data\\pdf_files\\Al Rouf.pdf', 'total_pages': 3, 'format': 'PDF 1.7', 'title': '', 'author': 'Saad Fahim', 'subject': '', 'keywords': '', 'moddate': '2025-10-31T16:05:18+05:30', 'trapped': '', 'modDate': "D:20251031160518+05'30'", 'creationDate': "D:20251031160518+05'30'", 'page': 0, 'source_file': 'Al Rouf.pdf', 'source_typr': 'pdf'}


[Document(metadata={'producer': 'Microsoft® Word for Microsoft 365', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2025-10-31T16:05:18+05:30', 'source': '..\\data\\pdf_files\\Al Rouf.pdf', 'file_path': '..\\data\\pdf_files\\Al Rouf.pdf', 'total_pages': 3, 'format': 'PDF 1.7', 'title': '', 'author': 'Saad Fahim', 'subject': '', 'keywords': '', 'moddate': '2025-10-31T16:05:18+05:30', 'trapped': '', 'modDate': "D:20251031160518+05'30'", 'creationDate': "D:20251031160518+05'30'", 'page': 0, 'source_file': 'Al Rouf.pdf', 'source_typr': 'pdf'}, page_content="Al Rouf Lighting Technology Co. Ltd. \nOfficial Company Profile (Internal Document) \nPage 1: About Al Rouf & Our Identity \nOur Company \nEstablished in 2014, Al Rouf Lighting Technology Co. Ltd. is a leading, ISO 9001-2015 QMS \nand ISO 45001:2018 HSE certified company based in the Kingdom of Saudi Arabia. We are a \npremier provider of comprehensive, turnkey lighting solutions. \nOur commitment to quality and innova

### Embeddings and vectorStoreDB

In [5]:
%pip install sentence-transformers chromadb scikit-learn


Note: you may need to restart the kernel to use updated packages.


c:\Users\saadf\Desktop\RAG-Youtube_Course\.venv\Scripts\python.exe: No module named pip


In [14]:
pip install -U sentence-transformers

Note: you may need to restart the kernel to use updated packages.


c:\Users\saadf\Desktop\RAG-Youtube_Course\.venv\Scripts\python.exe: No module named pip


In [15]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity


ModuleNotFoundError: No module named 'sentence_transformers'